In [ ]:
import pandas as pd
import numpy as np
import scipy
import pysr
import sympy
import math
from pysr import PySRRegressor
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle

In [ ]:
#Need to run on the command line
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
pip install sxs

In [ ]:
import sxs
print(sxs.__file__)
print(sxs.__version__)
print(dir(sxs))

In [ ]:
def dataframe(non_eccentric, non_spinning, aligned_spin, not_deprecated):
    df = sxs.load("dataframe")
    df = df.loc[np.isfinite(df["common_horizon_time"])] #This gets rid of any simulations with neutron stars
    df = df.loc[np.isfinite(df["reference_eccentricity"])] #Apparently some bhbh simulations with NaN values for the eccentricity
    if non_eccentric:
        df = df.loc[df['reference_eccentricity'] < 0.01]
    if non_spinning:
        df = df.loc[df["reference_dimensionless_spin1_mag"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_mag"] < 0.001]
    if aligned_spin:
        df = df.loc[df["reference_dimensionless_spin1_x"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_x"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin1_y"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_y"] < 0.001]
    if not_deprecated:
        df = df.loc[df["deprecated"] == False]
    #df = df.drop('SXS:BBH:0621') #All of my best models were having a hard time fitting CHT for this simulation. Seems like an outlier
    return df    
    
df = dataframe(non_eccentric = True, non_spinning = False, aligned_spin = False, not_deprecated = True)
print(len(df))

In [ ]:
ref_orb_period = 2*np.pi/df["reference_orbital_frequency_mag"]
CHT = df["common_horizon_time"] - df["reference_time"]

init_param_q = np.column_stack((ref_orb_period, df["reference_mass_ratio"]))

#This second block subtracts spin corrections from the quadrupolar, Newtonian model
init_param_spin = np.column_stack((ref_orb_period, df["reference_mass_ratio"], df["reference_chi_eff"]))

#This third block subtracts precession corrections from the quadrupolar, Newtonian model
init_param_precess = np.column_stack((ref_orb_period, df["reference_mass_ratio"], df["reference_chi2_perp"], df["reference_chi1_perp"]))
with open("chi_spin_model_0.749.pk", 'rb') as file:
    loaded_model_precess = pickle.load(file)

corrections = loaded_model_q.predict(init_param_q) + loaded_model_spin.predict(init_param_spin) + loaded_model_precess.predict(init_param_precess)

corrected_CHT = Newtonian_CHT([ref_orb_period, df["reference_mass_ratio"]]) - corrections #Subtracts "post-Newtonian" corrections from the Newtonian predictions for CHT

per_CHT_residual = 100*(corrected_CHT - CHT)/CHT #Defines whatever residual is left over after post-Newtonian corrections are applied to the Newtonian model

In [1]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
fig, ax = plt.subplots()

mpl.rcdefaults()
#I might need these next two lines in case trying to fix auto-bolded math text causes problems for some reason
font = {'family': 'sans-serif'}
plt.rc('font',**font)

#This block of code effectively makes all the text, markers, ticks, etc. on my plot large enough to be seen when I make the figure large
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 20
#mpl.rcParams['figure.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['legend.markerscale'] = 4
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
plt.tick_params(width = 2, length = 5)

# Make a plot with major ticks that are multiples of 20 and minor ticks that are multiples of 5. Label major ticks with '.0f' formatting but don't label
# minor ticks.  The string is used directly, the `StrMethodFormatter` is created automatically.
ax.xaxis.set_major_locator(MultipleLocator(4))
ax.xaxis.set_major_formatter('{x:.0f}')

# For the minor ticks, use no labels; default NullFormatter.
ax.xaxis.set_minor_locator(MultipleLocator(1))

#This plots all of the percent residuals against the corresponding mass ratios for the simulations
plt.scatter(df["reference_mass_ratio"], per_CHT_residual, marker = ".")

#I use these next two lines to identify the point with the highest error in the symbolic regression model
i = abs(per_CHT_residual).argmax()
plt.scatter(df["reference_mass_ratio"].iloc[i], per_CHT_residual.iloc[i], marker = ".", label = "Max Error")
print(i)

plt.axhline(y = 0, linestyle = '--', color = 'black')
plt.xlabel("Mass Ratio")
plt.ylabel("Error in CHT (%)")
plt.legend()

fig.set_size_inches(5, 5)
#"tight" makes sure that my figures don't get cut off when I save the figure
#plt.savefig("q_error.png", dpi=300, bbox_inches = "tight")
plt.show()

NameError: name 'plt' is not defined

In [ ]:

#The following two lines of code happen to throw a lot of errors, but they also seem to solve the problem I was having with the LaTeX axes labels
#appearing to be bolded for no reason
font = {'family':'serif','size':16, 'serif': ['computer modern roman']}
plt.rc('font',**font)

fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MultipleLocator(0.5))

graph = plt.scatter(df["reference_chi_eff"], per_CHT_residual, marker = ".")

i = abs(per_CHT_residual).argmax()
plt.scatter(df["reference_chi_eff"].iloc[i], per_CHT_residual.iloc[i], marker = ".")
plt.axhline(y = 0, linestyle = '--', color = 'black')

plt.xlabel(r'$\chi_{eff}$')

#Here I set some tick parameters, including hiding the ticks on the y-axis by coloring them white against the standard white matplotlib background
plt.yticks(color = 'w')
plt.tick_params(width = 2, length = 5)

fig = plt.gcf()
fig.set_size_inches(5, 5)
#plt.savefig("chi_error.png", dpi=300, bbox_inches = "tight")
plt.show()

In [ ]:
CHT_norm = CHT/np.max(CHT)
period_norm = ref_orb_period/np.max(ref_orb_period)
q_norm = df["reference_mass_ratio"]/np.max(df["reference_mass_ratio"])

simulations = np.column_stack((period_norm, q_norm, df["reference_chi_eff"], df["reference_chi1_perp"], df["reference_chi2_perp"]))

In [ ]:
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
def weighted_E(outputs, labels):
   return (outputs - labels)/labels #For every simulation in a batch, I take the difference between the CHT proportion
    #associated with that simulation and the CHT proportion predicted by my model weighted by the simulation proportion. This should give the proportion
    #residual of my model's CHT predictions, then I sum up over the batch and divide by the size of the batch
    
criterion = weighted_E

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self, activation_function):
        super().__init__()
        self.fc1 = nn.Linear(5, 120)
        self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 1)
        self.activation_function = activation_function
    
    def forward(self, x):
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
net = Net(nn.ReLU())
model_path = 'nnet_33177.pth'

net.load_state_dict(torch.load(model_path, weights_only=False))
net.eval()

CHT_norm = torch.tensor(CHT_norm, dtype=torch.float)
simulations = torch.tensor(simulations, dtype=torch.float)

class CustomDataset():
    def __init__(self, labels, data):
        self.labels = labels
        self.data = data

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sim = self.data[idx]
        label = self.labels[idx]
        return sim, label

eval_sims = CustomDataset(CHT_norm, simulations)

from torch.utils.data import DataLoader

batch_num = 1
eval_dataloader = DataLoader(eval_sims, batch_size=batch_num, shuffle=False)

eval_losses = np.zeros(len(df))

with torch.no_grad():
    for j, data in enumerate(eval_dataloader):
        inputs, labels = data

        inputs = inputs.float()
        
        outputs = net(inputs)

        outputs = outputs.squeeze()

        outputs = outputs.float()

        labels = labels.float()

        loss = criterion(outputs, labels)
        eval_losses[j] = 100 * loss.item()

In [ ]:
fig, ax = plt.subplots()

mpl.rcdefaults()
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 20
#mpl.rcParams['figure.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['legend.markerscale'] = 4
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
plt.tick_params(width = 2, length = 5)

ax.xaxis.set_major_locator(MultipleLocator(4))
ax.xaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_major_locator(MultipleLocator(10))
ax.yaxis.set_major_formatter('{x:.0f}')

plt.scatter(df["reference_mass_ratio"], eval_losses, marker = ".")
i = abs(eval_losses).argmax()
plt.scatter(df["reference_mass_ratio"].iloc[i], eval_losses[i], marker = ".", label = "Max Error")
print(i)
plt.axhline(y = 0, linestyle = '--', color = 'black')
plt.xlabel("Mass Ratio")
plt.legend()
plt.ylabel("Error in CHT (%)")

fig.set_size_inches(5, 5)
#"tight" makes sure that my figures don't get cut off when I save the figure
#plt.savefig("q_error_nn.png", dpi=300, bbox_inches = "tight")
plt.show()

In [ ]:
font = {'family':'serif','size':16, 'serif': ['computer modern roman']}
plt.rc('font',**font)

fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.yaxis.set_major_locator(MultipleLocator(10))

graph = plt.scatter(df["reference_chi_eff"], eval_losses, marker = ".")
i = abs(per_CHT_residual).argmax()
plt.scatter(df["reference_chi_eff"].iloc[i], eval_losses[i], marker = ".")
plt.axhline(y = 0, linestyle = '--', color = 'black')
plt.xlabel(r'$\chi_{eff}$')
plt.yticks(color = 'w')
plt.tick_params(width = 2, length = 5)
fig = plt.gcf()
fig.set_size_inches(5, 5)
#plt.savefig("chi_error_nn.png", dpi=300, bbox_inches = "tight")
plt.show()

In [ ]:
Newton_CHT_err = 100*(Newtonian_CHT([ref_orb_period, df["reference_mass_ratio"]]) - CHT)/CHT

reference_nums = np.zeros(len(df))
for i in range(len(df)):
    df.index[i]
    reference_nums[i] = int(df.index[i][-4:])

In [ ]:
fig, ax = plt.subplots()

mpl.rcdefaults()
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['axes.labelsize'] = 30
#mpl.rcParams['figure.labelsize'] = 30
mpl.rcParams['legend.fontsize'] = 22.5
mpl.rcParams['legend.markerscale'] = 6
mpl.rcParams['xtick.labelsize'] = 30
mpl.rcParams['ytick.labelsize'] = 30
plt.tick_params(width = 3, length = 8)

ax.xaxis.set_major_locator(MultipleLocator(1000))
ax.xaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_major_locator(MultipleLocator(40))
ax.yaxis.set_major_formatter('{x:.0f}')

plt.scatter(reference_nums, abs(Newton_CHT_err), marker = ".", label = "Newtonian")
plt.scatter(reference_nums, abs(eval_losses), marker = ".", label = "Neural Net")
plt.scatter(reference_nums, abs(per_CHT_residual), marker = ".", label = "PySR")
plt.yscale('log')
#plt.title("CHT Modeling Error Comparison")
plt.xlabel("Simulation Reference Number")
plt.ylabel("Absolute Error in CHT (%)")
plt.legend()

fig.set_size_inches(20, 10)
plt.savefig("q_error_Newt.png", dpi=300, bbox_inches = "tight")
plt.show()